# Examen de cómputo matricial equipo SVD

**Fecha**

19 de Abril de 2020

**Objetivo**

Programar en el lenguaje R el método de eliminación por bloques para la solución de un sistema de ecuaciones $$Ax=b$$ mediante el método de descomposición en valores singulares (DVS o SVD por siglas en inglés), a través del Algoritmo One-Sided Jacobi.

**Especificaciones de ambiente común de trabajo**

Para trabajar y a efecto de tener un entorno común de trabajo para el desarrollo del proyecto, por favor emplear la imagen de docker basada en R del curso MNO 2020 (palmoreck/jupyterlab_r_kernel:1.1.0)

```
docker run --rm -v `pwd`:/datos --name jupyterlab_r_kernel_local -p 8888:8888 -d palmoreck/jupyterlab_r_kernel:1.1.0

```

*Nota:* el comando "-v \`pwd\`:/datos", permite montar el directorio actual en donde el usuario se encuentre situada en terminal como un volumen de la imagen de docker, dentro del directorio "/datos".

**Comentarios**

Para mayor información consultar el [Project Board](https://github.com/mno-2020-gh-classroom/ex-modulo-3-comp-matricial-svd-czammar/projects/1), la [especificación simplificada del algoritmo](https://github.com/mno-2020-gh-classroom/ex-modulo-3-comp-matricial-svd-czammar/blob/master/References/Simplified_SVD_OneSidedJacobi_Algorithm.md) así como las instrucciones de los issues correspondientes


## 1. Funciones auxiliares

Las siguientes funciones serán procedimientos de apoyo en el diseño del método de eliminación por bloques para la solución de un sistema lineal mediante el método SVD.

### 1.1 Generación de índices

### 1.2 Verificación de ortogonalidad entre vectores

### 1.3 Función signo

In [1]:
signo<-function(x) {
  # Indica el signo de un número
  # Args: 
  #    X (numeric): número a revisar
  #Returns:
  #    TRUE si el número es positivo o cero
  #    FALSE si el número es negativo
  
  ifelse(x<0,FALSE,TRUE)
  }

### 1.4 Solver dada descomposición SVD

## 2. Algoritmo SVD y solución de sistema lineal

Esta sección aborda el diseño del método de eliminación por bloques para la solución de un sistema lineal mediante el método SVD.

### 2.1 One-sided Jacobi numerical aproximación

### 2.2 Linear solver aproximating SVD decomposition using One-sided Jacobi algorithm